### SEALION-v3-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs zip unzip python-is-python3 -y

In [ ]:
!git lfs install

In [ ]:
# Runpod is less strict
!pip install --upgrade transformers safetensors sentencepiece huggingface-hub protobuf accelerate bitsandbytes tqdm openai backoff retrying ipykernel ipywidgets matplotlib cmake scikit-build-core setuptools

In [ ]:
import os, sys, subprocess

os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Reduce thread contention
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["GIT_LFS_SKIP_SMUDGE"] = "1"

In [ ]:
!git clone https://huggingface.co/aisingapore/Llama-SEA-LION-v3-8B-IT-GGUF LLM_MODELS_GGUF/Llama-SEA-LION-v3-8B-IT-GGUF

In [ ]:
!pip install llama-cpp-python==v0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [ ]:
!cd LLM_MODELS_GGUF/Llama-SEA-LION-v3-8B-IT-GGUF && git lfs pull --include "*Q4_K_M*"

In [ ]:
# venv_dir = "venv"
# kernel_name = "runner_pipeline"
# display_name = "Python (runner_pipeline venv)"

# #create venv only if it doesn't already exist
# pyvenv_cfg = os.path.join(venv_dir, "pyvenv.cfg")
# if not os.path.exists(pyvenv_cfg):
#     print("venv not found, creating", venv_dir)
#     subprocess.check_call([sys.executable, "-m", "venv", venv_dir])
# else:
#     print("venv already exists at", venv_dir)
# venv_python = os.path.join(venv_dir, "bin", "python")

# if not os.path.exists(venv_python):
#     raise FileNotFoundError(f"Couldn't find python inside venv at {venv_python}")

# print("Using venv python:", venv_python)

# #install/upgrade pip and packages into the venv
# packages = [
#     "transformers","safetensors","sentencepiece","huggingface-hub",
#     "accelerate","bitsandbytes","tqdm","openai","backoff","retrying",
#     "protobuf","matplotlib","ipykernel","ipywidgets"
# ]

# # upgrade pip first
# subprocess.check_call([venv_python, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])
# subprocess.check_call([venv_python, "-m", "pip", "install"] + packages)

# print("Installed packages into the venv.")

# #register the kernel (if already registered, ipykernel will usually overwrite/refresh)
# try:
#     subprocess.check_call([venv_python, "-m", "ipykernel", "install", "--user",
#                            "--name", kernel_name, "--display-name", display_name])
#     print("Registered Jupyter kernel:", display_name)
# except subprocess.CalledProcessError as e:
#     print("Kernel registration failed or already exists:", e)

# # 5) quick verification print
# print("Switch the notebook kernel to:", display_name)
# print("then 'import sys; print(sys.executable)' should show the venv python.")


In [ ]:
os.environ["LLM_BACKEND"] = "llamacpp"

snapshot_path = "/workspace/LLM_MODELS_GGUF/Llama-SEA-LION-v3-8B-IT-GGUF/Llama-SEA-LION-v3-8B-IT-Q4_K_M.gguf"
#snapshot_path = "LLM_MODELS_GGUF/Llama-SEA-LION-v3-8B-IT-GGUF/Llama-SEA-LION-v3-8B-IT-Q4_K_M.gguf"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

### If kernel doesnt recognize
LLM_MODEL=snapshot_path
LOCAL_MODEL_PATH=snapshot_path


In [ ]:
# #Test for Max Time generation stopping criteria
# from llm_backends import HFBackend
# hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
# print("Calling short test (3s max_time)...")
# res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
# print("Result length:", len(res))
# print(res[:1000])

# print("Calling short test (10s max_time)...")
# res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
# print("Result length:", len(res))
# print(res[:1000])

In [ ]:
# Commandline args universal
# MAX_NEW_TOKENS is purely for text generation count limit while max_position_embeddings is for context_length based on LLM config.json. !!! input_length + MAX_NEW_TOKENS shopuld be < context_length, otherwise LLM breaks. Llama 3 only has 8k context length/max_posiiton_embedding. SEALIONv3-LLama3-8B-IT uses ROPE, max_position_embeddings follows ROPE limit 131k, Qwen2.5-7B-IT has 32k context length, SahabatAIv1-LLama3-8B-IT has 8k context length.
# Counted the response for each steps in notebook output cell with tokens counter online, translations ~400 tokens, ~decomposition ~500 tokens, search_resolve ~700 tokens. 1200 didn't cut it, need more
# Change this every process (translate, decompose, search_resolve), different value is needed. Time is in seconds.
os.environ["LLM_WORKER_MAX_TIME"] = "300"
LLM_WORKER_MAX_TIME=300
os.environ["MAX_NEW_TOKENS"] = "2500"
MAX_NEW_TOKENS=2500
os.environ["BATCH_NUM"] = "1"
BATCH_NUM=1

print("LLM_WORKER_MAX_TIME =", os.environ["LLM_WORKER_MAX_TIME"])
print("MAX_NEW_TOKENS =", os.environ["MAX_NEW_TOKENS"])
print("BATCH_NUM =", os.environ["BATCH_NUM"])

In [ ]:
# Translation with original prompts
#!python translate_to_fol.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated_translation/v3/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Translation with modified prompts
#!python translate_to_fol.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated_translation/v3/prompts_modified --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Translation with refined prompts
#!python translate_to_fol.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated_translation/v3/prompts_refine --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Decomposition with refined prompts
#!python decompose_to_cnf.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file and_or_decomposer_refine --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated_decomposition/v2/prompts_refine --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with original prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver  --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS 

In [ ]:
# Logic Resolver with original prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file logic_resolver --save_path results_translated_search_resolve/prompts_original --model_name $LLM_MODEL --batch_num $BATCH_NUM --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v2/prompts_original --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with modified prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Logic Resolver with modified prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_modified  --save_path results_translated_search_resolve/prompts_modified --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS 

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v2/prompts_modified --model_name $LLM_MODEL

In [ ]:
# Logic Resolver with refined prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_refine --save_path results_translated_search_resolve/v3/prompts_refine --model_name $LLM_MODEL --batch_num 1 --negation False --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Logic Resolver with refined prompts
#!python search_resolve.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file logic_resolver_refine  --save_path results_translated_search_resolve/v3/prompts_refine --model_name $LLM_MODEL --batch_num 1 --negation True --search_round 10 --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_search_resolve/v3/prompts_refine --model_name $LLM_MODEL

In [ ]:
# # Naive prompting only requires True or False answer based on context
# os.environ["MAX_NEW_TOKENS"] = "600"
# MAX_NEW_TOKENS=600
# os.environ["BATCH_NUM"] = "1"
# BATCH_NUM=1

In [ ]:
# Solving with naive prompting only
#!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file naive_prompting --split dev --save_path results_translated_naive_prompting/prompts_normal --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompts_normal --model_name $LLM_MODEL --evaluation_method naive_prompting

In [ ]:
# Solving with naive prompting modified only
#!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_modified --split dev --save_path results_translated_naive_prompting/prompts_modified --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompts_modified --model_name $LLM_MODEL --evaluation_method naive_prompting

In [ ]:
# Solving with naive prompting full question only
#!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file naive_prompting --split dev --save_path results_translated_naive_prompting/prompts_normal_full --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS --prompts_mode full

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompts_normal_full --model_name $LLM_MODEL --evaluation_method naive_prompting

In [ ]:
# Solving with naive prompting with explanations reasoning after answer
#!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_explanations_after_answer --split dev --save_path results_translated_naive_prompting/prompt_explanations_after_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompt_explanations_after_answer --model_name $LLM_MODEL --evaluation_method naive_prompting

In [ ]:
# Solving with naive prompting with explanations reasoning before answer
#!python naive_prompting.py --data_path results_bahasa_translation --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file naive_prompting_explanations_before_answer --split dev --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated_naive_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --evaluation_method naive_prompting

## Ablation

In [ ]:
# Solving with fol prompting with explanations reasoning before answer
!python ablation_fol.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translations_only_prompting --save_path results_translated_fol_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Solving with cnf prompting with explanations reasoning before answer
!python ablation_cnf.py --data_path results_translated_decomposition/v2/prompts_refine --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file decompositions_only_prompting --save_path results_translated_cnf_prompting/prompt_explanations_before_answer --model_name $LLM_MODEL --batch_num $BATCH_NUM --max_new_tokens $MAX_NEW_TOKENS

In [ ]:
# Generate prolog from FOL form
!python generate_prolog.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --save_path results_translated_fol_prolog --model_name $LLM_MODEL --workers 8 --swipl_path "C:/Program Files/swipl/bin/swipl.exe" --overwrite

In [7]:
# Solving with prolog
!python run_prolog_and_collect.py --dataset_name ProntoQA --save_path results_translated_fol_prolog --model_name $LLM_MODEL --workers 8 --swipl_path "C:/Program Files/swipl/bin/swipl.exe"

Running 500 files -> appending to results_translated_fol_prolog\ProntoQA\workspace_LLM_MODELS_GGUF_Llama-SEA-LION-v3-8B-IT-GGUF_Llama-SEA-LION-v3-8B-IT-Q4_K_M.gguf_naive_prompting.json
Processed ProntoQA_1.pl
Processed ProntoQA_10.pl
Processed ProntoQA_100.pl
Processed ProntoQA_101.pl
Processed ProntoQA_102.pl
Processed ProntoQA_104.pl
Processed ProntoQA_105.pl
Processed ProntoQA_103.pl
Processed ProntoQA_106.pl
Processed ProntoQA_107.pl
Processed ProntoQA_109.pl
Processed ProntoQA_108.pl
Processed ProntoQA_11.pl
Processed ProntoQA_111.pl
Processed ProntoQA_113.pl
Processed ProntoQA_116.pl
Processed ProntoQA_112.pl
Processed ProntoQA_115.pl
Processed ProntoQA_110.pl
Processed ProntoQA_114.pl
Processed ProntoQA_118.pl
Processed ProntoQA_119.pl
Processed ProntoQA_120.pl
Processed ProntoQA_12.pl
Processed ProntoQA_117.pl
Processed ProntoQA_122.pl
Processed ProntoQA_121.pl
Processed ProntoQA_126.pl
Processed ProntoQA_123.pl
Processed ProntoQA_124.pl
Processed ProntoQA_125.pl
Processed Pron